In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D ,MaxPooling2D,Activation,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard , EarlyStopping , ReduceLROnPlateau
import pickle
import time

In [23]:
#importation des donnees
x=pickle.load(open("X.pickle","rb"))
y=pickle.load(open("Y.pickle","rb"))




In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42);

In [25]:
#normalisation des donnees :
x=x/255
print(x.dtype)
print(x.shape)
print(y.dtype)
print(y.shape)

float32
(18815, 80, 80, 1)
int32
(18815,)


In [26]:
#Testes des combinaison possible pour atteindre le mailleur valeur val_loss + val_accuracy avec tensorboard
#conv_layers=[1,2]
#layer_sizes=[32,64]
#dense_layers=[0,1,2]
#for dense_layer in dense_layers:
#    for layer_size in layer_sizes:
#       for conv_layer in conv_layers:
#            NAME = "{}-conv-{}-nodes-{}-Dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
#            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
#            model = Sequential()
#            model.add(Conv2D(layer_size,kernel_size=3))
#            model.add(Activation("relu"))
#            model.add(MaxPooling2D(pool_size=2))
#            for l in range(conv_layer-1):
#                model.add(Conv2D(layer_size,kernel_size=3))
#                model.add(Activation("relu"))
#                model.add(MaxPooling2D(pool_size=2))
#            model.add(Flatten())
#            for d in range(dense_layer):
#                model.add(Dense(layer_size))
#                model.add(Dropout(0.2))
#                model.add(Activation("relu"))
#
#            model.add(Dense(9))
#            model.add(Activation("softmax"))
#            model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#            model.fit(x,y,epochs=10,batch_size=64,validation_split=0.2 ,callbacks=[tensorboard])
#

In [27]:
#les 2 meilleurs combinaison sont :2-conv-64-nodes-1-Dense et 2-conv-32-nodes-2-Dense-1764614100
#tensorboard + Earlystopping:
NAME = "Model_1".format(int(time.time()))
#callbacks :
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
early_stop=EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)
reduce_lr=ReduceLROnPlateau(monitor="val_loss",factor=0.2,patience=2,verbose=1,min_lr=1e-6)
model_1 = Sequential()
model_1.add(Conv2D(32,kernel_size=3))
model_1.add(BatchNormalization())
model_1.add(Conv2D(32,kernel_size=3))
model_1.add(BatchNormalization())
model_1.add(Activation("relu"))
model_1.add(MaxPooling2D(pool_size=2))
model_1.add(Dropout(0.3))
#demain essai de faire 3 conv successive
model_1.add(Conv2D(64,kernel_size=3))
model_1.add(BatchNormalization())
model_1.add(Conv2D(64,kernel_size=3))
model_1.add(BatchNormalization())
model_1.add(Activation("relu"))
model_1.add(MaxPooling2D(pool_size=2))
model_1.add(Dropout(0.3))


model_1.add(Conv2D(64,kernel_size=3))
model_1.add(BatchNormalization())
model_1.add(Conv2D(64,kernel_size=3))
model_1.add(Activation("relu"))
model_1.add(MaxPooling2D(pool_size=2))
model_1.add(Dropout(0.3))


model_1.add(Flatten())
model_1.add(Dense(128))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.4))
model_1.add(Activation("relu"))

model_1.add(Dense(9))
model_1.add(Activation("softmax"))
model_1.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.fit(x_train,y_train,epochs=30,batch_size=64,validation_split=0.2 ,callbacks=[early_stop,tensorboard,reduce_lr])

Epoch 1/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 79s 462ms/step - accuracy: 0.3851 - loss: 1.8003 - val_accuracy: 0.3591 - val_loss: 2.2685 - learning_rate: 0.0010
Epoch 2/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 76s 463ms/step - accuracy: 0.5119 - loss: 1.4158 - val_accuracy: 0.5543 - val_loss: 1.3588 - learning_rate: 0.0010
Epoch 3/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 76s 462ms/step - accuracy: 0.5593 - loss: 1.2572 - val_accuracy: 0.5763 - val_loss: 1.2446 - learning_rate: 0.0010
Epoch 4/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 76s 461ms/step - accuracy: 0.6014 - loss: 1.1567 - val_accuracy: 0.5714 - val_loss: 1.2871 - learning_rate: 0.0010
Epoch 5/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 77s 464ms/step - accuracy: 0.6207 - loss: 1.0976 - val_accuracy: 0.6029 - val_loss: 1.1845 - learning_rate: 0.0010
Epoch 6/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 76s 463ms/step - accuracy: 0.6385 - loss: 1.0503 - val_accuracy: 0.6705 - val_loss: 0.9682 - learning_rate: 0.0010
Epoch 7/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 76s 461ms/step - accuracy: 0.6

In [28]:
#NAME = "Model_2".format(int(time.time()))
#tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
#model_2 = Sequential()
#model_2.add(Conv2D(32,kernel_size=3))
#model_2.add(BatchNormalization())
#model_2.add(Activation("relu"))
#model_2.add(MaxPooling2D(pool_size=2))
#
#model_2.add(Conv2D(64,kernel_size=3))
#model_2.add(Activation("relu"))
#model_2.add(MaxPooling2D(pool_size=2))
#
#model_2.add(Flatten())
#model_2.add(Dense(32))
#model_2.add(Dropout(0.2))
#model_2.add(Activation("relu"))
#
#
#model_2.add(Dense(32))
#model_2.add(Dropout(0.4))
#model_2.add(Activation("relu"))
#
#model_2.add(Dense(9))
#model_2.add(Activation("softmax"))
#model_2.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#model_2.fit(x_train,y_train,epochs=30,batch_size=64,validation_split=0.2 ,callbacks=[early_stop,tensorboard])